## Prepare the Environment

#### Import Required Library

In [6]:
import random
import shutil
import sys

import pandas as pd
import xml.etree.ElementTree as ET
from collections import Counter
from pathlib import Path

import yaml
import PIL
from PIL import Image
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import cv2
import torch
import torchvision
from torchvision import transforms
from torchvision.io import read_image
from torchvision.transforms.functional import to_pil_image
from torchvision.utils import draw_bounding_boxes, make_grid

#### Install and Import YOLO from ultralytics

In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [7]:
import ultralytics
from ultralytics import YOLO

#### Check the Version of Library

In [10]:
print("Platform:", sys.platform)
print("Python version:", sys.version)
print("==============================")
print("matplotlib version:", plt.matplotlib.__version__)
print("pandas version:", pd.__version__)
print("PyYAML version:", yaml.__version__)
print("PIL version : ", PIL.__version__)
print("CV2 version : ", cv2.__version__)
print("torch version : ", torch.__version__)
print("torchvision version : ", torchvision.__version__)
print("ultralytics version : ", ultralytics.__version__)

Platform: linux
Python version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
matplotlib version: 3.10.0
pandas version: 2.2.2
PyYAML version: 6.0.2
PIL version :  11.2.1
CV2 version :  4.11.0
torch version :  2.6.0+cu124
torchvision version :  0.21.0+cu124
ultralytics version :  8.3.156


#### Check GPU Availability

In [11]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"Using {device} device.")

Using cpu device.


#### Mounting Google Drive for Accessing Datasets

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
%cd '/content/drive/MyDrive/Colab Notebooks/datasets/'

/content/drive/MyDrive/Colab Notebooks/datasets


In [ ]:
!apt-get install tree

In [35]:
!tree dhaka_traffic_datasets --filelimit=10

dhaka_traffic_datasets
└── train dataset  [6006 entries exceeds filelimit, not opening dir]

1 directory, 0 files


## Exploring the Data

In [36]:
dhaka_image_dir = Path("dhaka_traffic_datasets", "train dataset")
print(dhaka_image_dir)

dhaka_traffic_datasets/train dataset


In [37]:
dhaka_files = list(dhaka_image_dir.iterdir())
dhaka_files[-5:]

[PosixPath('dhaka_traffic_datasets/train dataset/Pias (96).jpg'),
 PosixPath('dhaka_traffic_datasets/train dataset/Pias (91).xml'),
 PosixPath('dhaka_traffic_datasets/train dataset/Pias (92).xml'),
 PosixPath('dhaka_traffic_datasets/train dataset/Pias (95).jpg'),
 PosixPath('dhaka_traffic_datasets/train dataset/Pias (93).xml')]

Two types of files:

1. `.xml` files: These contain the annotations for the images.
2. `.jpg` files: These are the actual image files.

Each image typically has a corresponding XML file.

In [38]:
file_extension_counts = Counter(Path(file).suffix for file in dhaka_files)

for extension, count in file_extension_counts.items():
    print(f"Files with extension {extension}: {count}")

Files with extension .xml: 3003
Files with extension .jpg: 2844
Files with extension .JPG: 143
Files with extension .png: 12
Files with extension .jpeg: 2
Files with extension .PNG: 2
